# AMIE 实践

在这个项目中，我们使用[AMIE](https://hal-imt.archives-ouvertes.fr/hal-01699866/document)对知识图谱进行规则挖掘。

- Jupyter Notebook 对 multiprocessing 支持不友好，本notebook仅供教学使用，如果实际运行完整的项目请使用 `AMIE.py`

## AMIE原理回顾
### 规则
知识图谱中的规则可以用以下形式表示：
$$ B_1 \land B_2 \land ... \land B_n \Rightarrow H $$
其中$B_1 \land B_2 \land ... \land B_n$表示规则的body部分，有n个原子(atom)组成，$H$表示规则的head部分，由一个原子组成，每个原子A可以表示为$A = r(x,y)$的形式，其中$r$表示为原子包含的关系，$x,y$为变量。

本项目中AMIE学习的规则为所有规则中的一个子集，即闭环的联通的规则，也可以叫做路径规则。可表示为：
$$ r_1(x, z_1) \land r_2(z_1, z_2) \land ... \land r_n(z_{n-1}, y) \Rightarrow r(x,y) $$
我们可以将其简化表示为$\vec{B}\Rightarrow r_0(x,y)$。

如果规则中的所有变量替换为具体的实体并保证每个实例化后的atom都存在图谱中，这样规则实例化后的结果称为规则的一个grounding。

### 规则的几个统计指标
#### Support
对于规则$rule$, support为在知识图谱中规则的grounding的个数, 可表示为
$$ supp(\vec{B}\Rightarrow r_0(x,y)) = \#(x,y): \exists z_1, ..., z_n : \vec{B}\Rightarrow r_0(x,y)) = \#(x,y) $$
#### HC (head coverage)
头覆盖度是另一个评价规则的指标，定义如下：
$$ HC(\vec{B}\Rightarrow r_0(x,y)) = \frac{suppo(\vec{B}\Rightarrow r_0(x,y))}{size(r)} $$
其中$size(r)$表示图谱中关系存在的关于$r$的三元组数量，即$(x, r, y)$的数量。
#### Confidence
置信度的定义如下：
$$ conf(\vec{B}\Rightarrow r_0(x,y)) = \frac{suppo(\vec{B}\Rightarrow r_0(x,y))}{\#(x,y): \exists z_1, ..., z_n : \vec{B}} $$
#### PCA Confidence
PCA置信度全称为 partial closed world assumption confidence,即局部封闭世界假设。在封闭世界假设(closed world assumption)中，图谱中不存在的三元组都被视为错误的，而在局部封闭世界假设中假设不存在知识图谱中的三元组不一定是错误的，有的缺失是图谱本身的不完整性造成的。PCA confidence的计算方式如下：
$$ conf_{pca}(\vec{B}\Rightarrow r_0(x,y)) = \frac{suppo(\vec{B}\Rightarrow r_0(x,y))}{\#(x,y): \exists z_1, ..., z_n : \vec{B} \land r(x, y^\prime)} $$
与confidence的定义不同，PCA confidence只将满足规则body部分且存在三元组$(x, r, y^\prime)$的实例才计算入分母，由于考虑了知识图谱本身的不完整性，通常PCA confidence会比confidence更准确。

## AMIE算法
AMIE是一个基于搜索遍历和剪枝的规则挖掘算法，此类算法的重点在于剪枝策略。
下面我们通过代码进行解释，首先导入需要的依赖包，本demo采用了多进程的挖掘方式。

In [1]:
from multiprocessing import Queue, Process, Pool

from collections import defaultdict
from copy import deepcopy
import numpy as np
import time
from timeit import default_timer

AMIE中的超参包含了三个，一个是最小的头覆盖度minHC，规则的最大长度maxLen, 以及最小置信度minConf，这里的用的是PCA confidence，为了方便起见，后文如果不做特殊说明，confidence都代指PCA confidence。这三个超参是为了规则搜索空间的剪枝服务的。

In [2]:
start = default_timer()
data_dir = '../../data/AMIE/WN18RR/'
minHC = 0.001
maxLen = 311
minConf = 0.01
epsilon = 1e-5

定义两个队列，一个是用于存储候选规则的队列ruleQ，一个是用于存储挖掘出的规则的队列outputQ。

In [3]:
ruleQ = Queue()
outputQ = Queue()

定义一些在剪枝过程中需要使用的字典，都是关于图谱中的信息

In [4]:
head = defaultdict(set)
tail = defaultdict(set)
hr2t = defaultdict(set)
tr2h = defaultdict(set)
ht2r = defaultdict(set)
r2ht = defaultdict(set)
head2num = defaultdict(int)
tail2num = defaultdict(int)
asHeadOfRelation = defaultdict(set)
asTailOfRelation = defaultdict(set)

首先读取知识图谱中train.txt的信息，train.txt中每行为一个三元组，顺序为h r t并用tab符号隔开

在读取数据的过程中我们自动添加了关系r的逆关系r_inv, 并为每个三元组(h,r,t)构建了一个逆三元组(t, r_inv, h).

In [5]:
def read2id(dir, REL=None):
    item2id = dict()
    id2item = dict()
    items = []
    with open(dir, 'r') as f:
        for l in f.readlines():
            ll = l.strip().split('\t')
            assert len(ll) == 2
            item = ll[0]
            id = ll[1]
            item2id[item] = int(id)
            id2item[int(id)] = item
            items.append(int(id))
            if REL is not None:
                item2id[item+'_inv'] = int(id)+REL
                id2item[int(id)+REL] = item+'_inv'
                items.append(int(id)+REL)
    num_item = len(item2id)
    return item2id, id2item, num_item, items

def record_triple(h,r,t):
    head[r].add(h)
    tail[r].add(t)
    hr2t[(h,r)].add(t)
    tr2h[(t,r)].add(h)
    r2ht[r].add((h,t))
    ht2r[(h,t)].add(r)
    head2num[(r,h)] += 1

trp_dir = data_dir + 'train.txt'
ent2id, id2ent, num_ent, ents = read2id(data_dir + 'entity2id.txt')
rel2id, id2rel, num_rel, rels = read2id(data_dir + 'relation2id.txt', REL=11)
print('#ENT:%d 	#REL:%d'%(num_ent, num_rel))

line_num = 0
with open(trp_dir) as f:
    for l in f.readlines():
        line_num += 1
        h,r,t = l.strip().split('\t')
        h = ent2id[h]
        r = rel2id[r]
        t = ent2id[t]
        r_inv = r+num_rel
        record_triple(h,r,t)
        record_triple(t, r_inv, h)

#ENT:40943 	#REL:22


初始化规则队列，初始的规则为只包含head的规则，在程序中我们用列表\[$r, r_1, ..., r_n, 1$\]表示闭合规则$r_1(x, z_1) \land r_2(z_1, z_2) \land ... \land r_n(z_{n-1}, y) \Rightarrow r(x,y)$， 用\[$r, r_1, ..., r_n, 0$\]非闭合规则$ r_1(x, z_1) \land r_2(z_1, z_2) \land ... \land r_n(z_{n-1}, y^\prime) \Rightarrow r(x,y)$. 注意别表的最后一位数字用于表示规则是否闭合。初始化的规则均为非闭合规则。

In [6]:
for rel in id2rel.keys():
    ruleQ.put([rel, 0])

首先定义一些规则的学习指标函数，包括判断规则是否闭合，规则的长度，规则的support，PCA confidence以及HC指标。

In [7]:
def closed(rule):
    return rule[-1] == 1

def length(rule):
    return len(rule)-1

def support(rule, PCA=False):
    lenth = length(rule)
    close = closed(rule)
    if lenth == 2:
        r0 = rule[0]
        r1 = rule[1]
        if close:
            sup = len(r2ht[r0].intersection(r2ht[r1]))
        else:
            intersection_e = head[r0].intersection(head[r1])
            sup = 0
            for h in intersection_e:
                num1 = head2num[(r0,h)]
                num2 = head2num[(r1,h)]
                sup += num1*num2
        pca_num = np.sum(np.asarray([(len(hr2t[(h, r0)]) != 0) * head2num[(r1, h)] for h in head[r1]], dtype=np.int64))
    elif close and lenth ==3:
        r0 = rule[0]
        r1 = rule[1]
        r2 = rule[2]
        sup = sum([len(hr2t[(h, r1)].intersection(tr2h[(t,r2)])) for h,t in r2ht[r0]])
        pca_num = 0
        for h,t in r2ht[r1]:
            pca_num += np.sum(np.asarray([len(hr2t[(h, r0)])!=0 for tt in hr2t[(t,r2)]], dtype=np.int64))
    else:
        print(rule, length(rule))
        raise NotImplementedError('rule do not calculate support')
    if PCA:
        return sup, pca_num
    else:
        return sup

def pca_confidence(rule):
    sup, pca_num = support(rule, PCA=True)
    return (sup/pca_num)

def HC(rule):
    sup = support(rule)
    headr = rule[0]
    return sup/(len(r2ht[headr])+epsilon)

conf_increase函数用于判断一条规则的confidence是否在其父规则的基础上有增加

In [8]:
def conf_increase(rule):
    if length(rule) == 2:
        return True
    parent = deepcopy(rule)
    del parent[-2]
    if closed(rule): parent[-1] = 0
    parent_conf = pca_confidence(parent)
    child_conf = pca_confidence(rule)
    return child_conf>parent_conf

acceptedForOutput函数用于判断一条规则是否满足输出规则的条件，

以下规则不满足输出条件：（1）规则是非闭合的，（2规则的confidence小于设置的minConf，（3）规则的confidence在父规则的基础上没有增加

In [9]:
def acceptedForOutput(rule):
    if not closed(rule):
        return False
    elif pca_confidence(rule) < minConf:
        return False
    elif not conf_increase(rule):
        return False
    else:
        return True

addingDanglingAtom 和 addingClosingAtom 两个函数是规则refine阶段的两个操作

addingDanglingAtom在原有规则基础上增加一个atom，这个atom $r_i(x,y)$，其中$x$是规则中已存在的变量，$y$是规则中不存在的变量。

addingClosingAtom在原有规则基础上增加一个atom，这个atom $r_i(x,y)$，其中$x,y$都是规则中已存在的变量，并且添加后是的规则闭合。

In [10]:
def addingDanglingAtom(rule):
    assert rule[-1] != 1
    if length(rule) == 2:
        return []
    for rel in rels:
        rule_new = deepcopy(rule)
        rule_new.insert(-1, rel)
        if HC(rule_new) >= minHC:
            yield rule_new
def addingClosingAtom(rule):
    assert rule[-1] != 1
    for i, rel in enumerate(rels):
        rule_new = deepcopy(rule)
        rule_new.insert(-2, i)
        rule_new[-1] = 1
        if HC(rule_new) >= minHC:
            yield rule_new

定义多进程的任务task。

这个任务的定义主要包含以下操作：

(1) 从规则队列inputQ中获取一个候选规则。

(2) 通过acceptedForOutput函数判断这个规则是够满足输出的条件，如果满足，将其加入到输出规则队列。

(3) 如果规则还未达到maxLen并且为非闭合的，则对规则进行refine，包括addingDanglingAtom，和addingClosingAtom两种操作，并将所有refine过程中新生成的规则填加到规则队列中。

(4) 如果规则队列已经为空，结束当前规则挖掘进程。


In [11]:
def task(inputQ, outputQ):
    while True:
        if not inputQ.empty():
            rule = inputQ.get()
            if length(rule)!=1 and acceptedForOutput(rule): outputQ.put(rule)
            if length(rule)<maxLen and not closed(rule):
                for rul in addingDanglingAtom(rule):
                    inputQ.put(rul)
                for rul in addingClosingAtom(rule):
                    inputQ.put(rul)
        else:
            print('task done')
            break

规则挖掘完成对挖掘到的规则进行解析：

In [12]:
def decodeRules():
    num_rules = 0
    print('RULE 	SUPP	HC	CONF_pca')
    while not outputQ.empty():
        rul = outputQ.get()
        num_rules += 1
        assert closed(rul)
        rul_rs = deepcopy(rul)
        del rul_rs[-1]
        rule_str = ''
        for i,r in enumerate(rul[:-1]):
            r = id2rel[r]
            rule_str += r
            if i == 0:
                rule_str += ' <== '
            if i < length(rul)-1 and i != 0:
                rule_str += ' & '
        print('%s 	%.d	%.4f	%.4f'%(rule_str, support(rul),HC(rul), pca_confidence(rul)))
    print('%d rules are learned'%(num_rules))

定义多个规则挖掘进程，以task函数为目标，以ruleQ和outputQ为变量
notebook 不支持多进程，下面代码不能执行，请使用AMIE.py

In [13]:
# 这些代码不能执行，因为notebook不支持多进程，请移步AIME.py
# num_worker = 10
# for i in range(num_worker):
#     p = Process(target=task, args=(ruleQ, outputQ))
#     print('process ' + str(i) + ' start')
#     p.start()
#
# while True:
#     if ruleQ.empty():
#         print('Rule mining finished')
#         decodeRules()
#         end = default_timer()
#         print('with in %3f'%(end-start))
#         break
#     else:
#         time.sleep(3)
#         continue

## 后记

由于WN18RR是一个关系比较少的数据集，所以挖掘出的规则数量较少，同学们有兴趣可以自己采用其他数据集试一试，同时也可以探索一些不同的minHC和minConf的设置对算法运行的影响。

AMIE+采用了数据库查询语言SQL和SPARQL搭配数据库系统进行了算法增速，本项目中未涉及。同时项目中还省去了一些算法细节，有兴趣的同学可以自己阅读原文并查看其源码。